In [3]:
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()

X = np.random.uniform(size=(2, 20))
net(X)
print(net[1].params)
print(type(net[1].bias))
print(net[1].bias)
print(net[1].bias.data())
# class MLP(nn.Block):
#     # 用模型参数声明层。这里，我们声明两个全连接的层
#     def __init__(self, **kwargs):
#         # 调用MLP的父类Block的构造函数来执行必要的初始化。
#         # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
#         super().__init__(**kwargs)
#         self.hidden = nn.Dense(256, activation='relu')  # 隐藏层
#         self.out = nn.Dense(10)  # 输出层

#     # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
#     def forward(self, X):
#         return self.out(self.hidden(X))
    
    
# net = MLP()
# net.initialize()
# net(X)
# class MySequential(nn.Block):
#     def add(self, block):
#     # 这里，block是Block子类的一个实例，我们假设它有一个唯一的名称。我们把它
#     # 保存在'Block'类的成员变量_children中。block的类型是OrderedDict。
#     # 当MySequential实例调用initialize函数时，系统会自动初始化_children
#     # 的所有成员
#         self._children[block.name] = block

#     def forward(self, X):
#         # OrderedDict保证了按照成员添加的顺序遍历它们
#         for block in self._children.values():
#             X = block(X)
#         return X
   
# net = MySequential()
# net.add(nn.Dense(256, activation='relu'))
# net.add(nn.Dense(10))
# net.initialize()
# net(X)


class FixedHiddenMLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 使用get_constant函数创建的随机权重参数在训练期间不会更新（即为常量参数）
        self.rand_weight = self.params.get_constant(
            'rand_weight', np.random.uniform(size=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, X):
        X = self.dense(X)
        # 使用创建的常量参数以及relu和dot函数
        X = npx.relu(np.dot(X, self.rand_weight.data()) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.dense(X)
        # 控制流
        while np.abs(X).sum() > 1:
            X /= 2
        return X.sum()
  
# net = FixedHiddenMLP()
# net.initialize()
# net(X)


class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')

    def forward(self, X):
        return self.dense(self.net(X))

chimera = nn.Sequential()
chimera.add(NestMLP(), nn.Dense(20), FixedHiddenMLP())
chimera.initialize()
chimera(X)



dense13_ (
  Parameter dense13_weight (shape=(10, 256), dtype=float32)
  Parameter dense13_bias (shape=(10,), dtype=float32)
)
<class 'mxnet.gluon.parameter.Parameter'>
Parameter dense13_bias (shape=(10,), dtype=float32)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


array(0.5202446)